<a href="https://colab.research.google.com/github/Renjian-buchai/buildingBloCS/blob/main/csvtoTFRecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Object Detection API


In [72]:
# !unzip drive/MyDrive/ExtractedSet.zip
!pip install -q tensorflow-object-detection-api


Import Requirements

In [71]:
import pandas as pd
import tensorflow as tf
import json
from PIL import Image
import io
import os
from object_detection.utils import dataset_util

Convert CSVs to dataframes


In [73]:
df = pd.read_csv('/content/ExtractedSet/annotations.csv')
df2 = pd.read_csv('/content/ExtractedSet/annotations2.csv')
df3 = pd.read_csv('/content/ExtractedSet/annotations3.csv')

Create tf examples.

In [67]:
def create_tf(encoded_cat_image_data,height,width,filename,xmin,xmax,ymin,ymax):
  # height = height
  # width = width
  # filename = filename
  image_format = b'jpg'
  xmins = [xmin/ width]
  xmaxs = [xmax /width]
  ymins = [ymin / height]
  ymaxs = [ymax / height]
  classes_text = [b'Pill']
  classes = [1]
  filename = filename.encode()
  tf_example = tf.train.Example(features=tf.train.Features(feature={
  'image/height': dataset_util.int64_feature(height),
  'image/width': dataset_util.int64_feature(width),
  'image/filename': dataset_util.bytes_feature(filename),
  'image/source_id': dataset_util.bytes_feature(filename),
  'image/encoded': dataset_util.bytes_feature(encoded_image_data),
  'image/format': dataset_util.bytes_feature(image_format),
  'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
  'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
  'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
  'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
  'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
  'image/object/class/label': dataset_util.int64_list_feature(classes),
    }
  ))
  return tf_example

Function to convert csv from VGG image annotator to TFRecord file for training

In [70]:
output_file = 'training.tfrecord'
writer = tf.io.TFRecordWriter(output_file)

for i in range(len(df)):
  #encoded_image_data
  directory_path = "/content/ExtractedSet"
  filename = df.loc[i, "filename"]
  full_file_path = os.path.join(directory_path, filename)
  image = Image.open(full_file_path)
  # with open(full_file_path, 'rb') as f:
  #   image_bytes = f.read()
  # encoded_image_data = bytes(image_bytes)
  image_buffer = io.BytesIO()
  image.save(image_buffer, format='JPEG')
  # print(encoded_image_data)
  encoded_image_data = (image_buffer.getvalue())
  
  #height and width
  json_string = df.loc[i, "region_shape_attributes"]
  json_object = json.loads(json_string)
  width = json_object["width"]
  height = json_object["height"]
  #filename = filename
  # df.loc[i, "filement"]
  #x_min,x_max,y_min,y_max
  x_min = df2.loc[i, "x_min"]
  x_max = df2.loc[i, "x_max"]
  y_min = df2.loc[i, "y_min"]
  y_max = df2.loc[i, "y_max"]
  filename = os.path.splitext(filename)[0]
  example = create_tf(encoded_image_data,height,width,filename,x_min,x_max,y_min,y_max)
  serialized_example = example.SerializeToString()
  writer.write(serialized_example)
writer.close()
